## Data Preprocessing

In [1]:
import os.path
import urllib.parse
import pandas as pd
import json

import requests
import sqlite3


from thrift.transport import TSocket
import socket
from thrift.transport import TTransport

import numpy as np
import re
import math
from scipy import stats

In [2]:
from bs4 import BeautifulSoup

In [3]:
from sklearn.impute import KNNImputer

In [4]:
filename=pd.read_csv("train(1).csv")
filename.head(10000)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


##read file

In [5]:
def check_flatfile(filename):
    extension = os.path.splitext(filename)[1][1:]
    return extension

def read_file(filename=None,source=None,dbtype=None,queryfile=None,hostip=None,portno=None,username1=None,passwrd=None,dbname=None):
    try:
        if source=='flatfile' or source=='url':
            y=check_flatfile(filename)
            if y=='json':
                with open(filename,'r') as json_file:
                    data=json.load(json_file)
                    return(pd.DataFrame(data))

            elif y=='csv':   
                data=pd.read_csv(filename,sep=';')
                return(data)

            elif y=='txt':
                data=pd.read_csv(filename,sep='\s+')
                return(data)

            elif y=='xml':
                r=requests.get(filename)
                xml_doc=r.text
                df=BeautifulSoup(xml_doc,"html5lib")
                data=df.prettify()
                print("This is Not a DataFrame!!")
                return(data)
                
            else:
                print("The Following File Format is Not Correct!!")
                
        elif source=='database':
            
            if dbtype=='sqlite':
                conn = sqlite3.connect(filename) #It connects to the database server.
                c = conn.cursor()
                with open(queryfile,'r') as inserts:
                    for statement in inserts:
                        c.execute(statement)
                data=c.fetchall()
                return(pd.DataFrame(data))
                conn.commit()
                conn.close()
                
            elif dbtype=='mysql'or dbtype=='postgresql' or dbtype=='oracle' or dbtype=='sqlserver' or dbtype=='teradata' or dbtype=='netezza':
                
                if(hostip!=None and portno!=None and username1!=None and passwrd!=None and dbname!=None):
                    try:
                        conn = pymysql.connect(host=hostip, port=portno, user=username1, password=passwrd, db=dbname)
                        c = conn.cursor()
                        with open(queryfile,'r') as inserts:
                            for statement in inserts:
                                c.execute(statement)
                        data=c.fetchall()
                        return(pd.DataFrame(list(data)))
                        conn.commit()
                        conn.close()
                        
                    except:
                        print("Entered Database Settings is Not Correct!!")
                        
            elif dbtype=='hive':
                
                if(hostip!=None and portno!=None and username1!=None and passwrd!=None and dbname!=None):
                    try:
                        conn = hive.connect(host=hostip, port=portno, username=username1, password=passwrd, database=dbname)
                        c = conn.cursor()
                        with open(queryfile,'r') as inserts:
                            for statement in inserts:
                                c.execute(statement)
                        data=c.fetchall()
                        return(pd.DataFrame(list(data)))
                        conn.commit()
                        conn.close()
                        
                    except:
                        print("Entered Database Settings is Not Correct!!")
                        
                else:
                    print("Please Enter all the Database Settings!!")
     
            else:
                print("Enter a Valid Database Type!!")   
        
        else:
            print("Please Enter a Valid File Type!!")
            
    except FileNotFoundError:
        print("File Not Found!!")

In [6]:
#Missing Value Treatment
        
def dist_count(dataset):
    value_counts_with_nan = dataset.value_counts(dropna=False)
    value_counts_without_nan = value_counts_with_nan.loc[value_counts_with_nan.index.dropna()]
    distinct_count_with_nan = value_counts_with_nan.count()
    
    if value_counts_without_nan.index.inferred_type == "mixed":
        raise TypeError('Not supported mixed type')
    
    result = distinct_count_with_nan
    return(result)

def var_type(filepath):
    
    data= pd.read_csv(filepath)
    vartype=pd.DataFrame(columns = ['name','type_col'])
    for column in data:
    
        distinct_count = dist_count(data[column])
        leng = len(data[column])

        if distinct_count <= 1:
            vartype=vartype.append({'name': column,'type_col':'Constant'},ignore_index=True)
        elif pd.api.types.is_bool_dtype(data[column]) or (distinct_count == 2 and pd.api.types.is_numeric_dtype(data[column])):
            vartype=vartype.append({'name': column,'type_col':'Bool'},ignore_index=True)
        elif pd.api.types.is_numeric_dtype(data[column]):           
            if (((distinct_count/leng)*100)>=20):
                vartype=vartype.append({'name': column,'type_col':'Numeric Continuous'},ignore_index=True)
            else:
                vartype=vartype.append({'name': column,'type_col':'Numeric Categorical'},ignore_index=True)
        elif pd.api.types.is_datetime64_dtype(data[column]):
            vartype=vartype.append({'name': column,'type_col':'Date'},ignore_index=True)
        elif distinct_count == leng:
            vartype=vartype.append({'name': column,'type_col':'Unique'},ignore_index=True)
        else:
            vartype=vartype.append({'name': column,'type_col':'Others'},ignore_index=True)

    return(vartype)

def vari_type(filepath):
    
    data= pd.read_csv(filepath)
    vartype=pd.DataFrame(columns = ['name','type_col'])
    for column in data:
    
        distinct_count = dist_count(data[column])
        leng = len(data[column])

        if distinct_count <= 1:
            vartype=vartype.append({'name': column,'type_col':'continuous'},ignore_index=True)
        elif pd.api.types.is_bool_dtype(data[column]) or (distinct_count == 2 and pd.api.types.is_numeric_dtype(data[column])):
            vartype=vartype.append({'name': column,'type_col':'continuous'},ignore_index=True)
        elif pd.api.types.is_numeric_dtype(data[column]):
            vartype=vartype.append({'name': column,'type_col':'continuous'},ignore_index=True)
        elif pd.api.types.is_datetime64_dtype(data[column]):
            vartype=vartype.append({'name': column,'type_col':'continuous'},ignore_index=True)
        elif distinct_count == leng:
            vartype=vartype.append({'name': column,'type_col':'continuous'},ignore_index=True)
        else:
            vartype=vartype.append({'name': column,'type_col':'categorical'},ignore_index=True)

    return(vartype)
        

def missing_value_treatment(source=None,imputation='knn',nearest_row=None,treatment=None,custom=None):
    
    dataset= pd.read_csv(source,header=0)
    NaN=dataset.isnull().values.any()

    if NaN==True:
        
        if imputation=='knn':
            treated_dataset= pd.DataFrame(KNN(k=nearest_row).complete(dataset))
            return(treated_dataset)
        
        elif imputation=='columnwise':
            dtype= vari_type(source)  #It is used to check if a column is categorical or continuous.
            
            for index, column in dtype.iterrows():
                
                if column['type_col']=='categorical':

                    if treatment==None or treatment=='mode': #Default treatment is 'Mode' for categorical columns.
                        dataset[column['name']]= dataset[column['name']].fillna(dataset[column['name']].mode())

                    elif treatment=='delete':
                        dataset= dataset.dropna(inplace=False)

                    elif treatment=='custom':
                        dataset[column['name']]= dataset[column['name']].fillna(custom)

                    elif treatment=='nothing':
                        dataset[column['name']]= dataset[column['name']]

                elif column['type_col']=='continuous':  

                    if treatment==None or treatment=='mean':  #Default treatment is 'Mean' for continuous columns.
                        dataset[column['name']]= dataset[column['name']].fillna(dataset[column['name']].mean())

                    elif treatment=='delete':
                        dataset= dataset.dropna(inplace=False)

                    elif treatment=='mode':
                        dataset[column['name']]= dataset[column['name']].fillna(dataset[column['name']].mode())

                    elif treatment=='median':
                        dataset[column['name']]= dataset[column['name']].fillna(dataset[column['name']].median())

                    elif treatment=='custom':
                        dataset[column['name']]= dataset[column['name']].fillna(custom)

                    elif treatment=='nothing':
                        dataset[column['name']]=dataset[column['name']]

                else:
                    print("Unsupported Database!!!")
                    
            return(dataset)
                    
        else:
            print("Please Enter a Correct Imputation!!")
                      
    else:
        print("The Dataset does not contain any Missing Values!!")

In [7]:
def uniform(source,column): #This function returns/ Non-Uniformly Spaced /Uniformly Distributed.
    
    data=pd.read_csv(source,header=0)
    x=data[column].unique()
    x.sort()
    temp=pd.DataFrame(x)
    dif=temp.diff().dropna()
    mean=dif.mean()
    std=dif.std()
    uniform=0
    count=0
    uni=0
    nonuni=0
    for index, row in dif.iterrows():
        count=count+1
        if (row <= (mean+(1.5*std))).bool() and (row >= (mean-(1.5*std))).bool():
            uni=uni+1
        else:
            nonuni=nonuni+1

    floor=math.floor((count*85)/100)

    if uni<=floor:
        test1='Uniformly Spaced'
    else:
        test1='Non-Uniformly Spaced'
    
    df=data[column].dropna()
    total=0
    for i in range(10):
        dt=df.sample(30)
        x=stats.shapiro(dt)
        if x[1]>= 0.05:
            total=total+1
    
    if total>=6:
        test1='Uniformly Distributed'

    return(test1)
        

def check(test_str): #Check the string is only text

    pattern = r'[^\.a-z A-Z]'
    if re.search(pattern, test_str):
        #Character other then . a-z/A-Z was found
        return 1
    else:
        #No character other then . a-z/A-Z was found
        return 0
    
def check_text(source,column): #Returns the column is only text or not.
    df= pd.read_csv(source,header=0)
    count=df[column].apply(lambda x:check(x) if pd.notnull(x) else x).sum()
    if count==0:
        return('Only Text')
    else:
        return('Text with Number & Special Character')

def dist_count(dataset): #find total distinct value in a column
    value_counts_with_nan = dataset.value_counts(dropna=False)
    value_counts_without_nan = value_counts_with_nan.loc[value_counts_with_nan.index.dropna()]
    distinct_count_with_nan = value_counts_with_nan.count()
    
    if value_counts_without_nan.index.inferred_type == "mixed":
        raise TypeError('Not supported mixed type')
    
    result = distinct_count_with_nan
    return(result)

def check_inputdata_attributes_column_type(filepath):
    
    data= pd.read_csv(filepath)
    vartype=pd.DataFrame(columns = ['Name','Type_col'])
    for column in data:
    
        distinct_count = dist_count(data[column])
        leng = len(data[column]) 
        
        
        if distinct_count <= 1: #Check for constant column
            vartype=vartype.append({'Name': column,'Type_col':'Constant'},ignore_index=True)
            
        elif pd.api.types.is_bool_dtype(data[column]) or (distinct_count == 2): #Check boolean column
            vartype=vartype.append({'Name': column,'Type_col':'Boolean'},ignore_index=True)
        
        elif pd.api.types.is_numeric_dtype(data[column]): #check for numeric column
            if len(str(data[column].max()))==1:
                vartype=vartype.append({'Name': column,'Type_col':'Numeric With Length=1'},ignore_index=True)
                
            elif (((distinct_count/leng)*100)>=20):
                vartype=vartype.append({'Name': column,'Type_col':'Numeric Continuous'},ignore_index=True)
                
            elif uniform(filepath,column):
                vartype=vartype.append({'Name': column,'Type_col':uniform(filepath,column)},ignore_index=True)
                                                      
        elif pd.api.types.is_datetime64_dtype(data[column]): #check date type column
            vartype=vartype.append({'Name': column,'Type_col':'Date'},ignore_index=True)
                  
        elif check_text(filepath,column):
            vartype=vartype.append({'Name': column,'Type_col':check_text(filepath,column)},ignore_index=True)

        else:
            vartype=vartype.append({'Name': column,'Type_col':'Not Known'},ignore_index=True)
                
    return(vartype)

In [8]:
filename.head(10000)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [9]:
samm=pd.read_csv('train(1).csv')


In [10]:
samm.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
samm

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [12]:
var_type('train(1).csv')

,name,type_col
0,PassengerId,Numeric Continuous
1,Survived,Bool
2,Pclass,Numeric Categorical
3,Name,Unique
4,Sex,Others
5,Age,Numeric Categorical
6,SibSp,Numeric Categorical
7,Parch,Numeric Categorical
8,Ticket,Others
9,Fare,Numeric Continuous


In [13]:
check_inputdata_attributes_column_type('train(1).csv')

,Name,Type_col
0,PassengerId,Numeric Continuous
1,Survived,Boolean
2,Pclass,Numeric With Length=1
3,Name,Text with Number & Special Character
4,Sex,Boolean
5,Age,Uniformly Distributed
6,SibSp,Numeric With Length=1
7,Parch,Numeric With Length=1
8,Ticket,Text with Number & Special Character
9,Fare,Numeric Continuous


In [14]:
samm.head(1000000)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [15]:
missing_value_treatment(source='train(1).csv',imputation='columnwise',nearest_row='887',treatment='Mean',custom=None)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [16]:
samm.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [17]:
samm.dropna(subset=['Age'],inplace=True)

In [18]:
samm['Cabin']

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
885     NaN
886     NaN
887     B42
889    C148
890     NaN
Name: Cabin, Length: 714, dtype: object

In [19]:
samm.dropna(subset=['Cabin'],inplace=True)
samm.dropna(subset=['Embarked'],inplace=True)

In [20]:
samm.to_csv('pairwise.csv')

In [21]:
abcd=pd.read_csv('pairwise.csv')
abcd.isnull().sum()

Unnamed: 0     0
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64